In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.spatial import cKDTree
from shapely.geometry import Point, LineString, Polygon
from shapely.ops import nearest_points
from shapely import affinity

In [ ]:
# On-From-To list file name (streets to be mapped)
oft_fn = "N:/GIS/_2023/Projects/OnFromTo Mapping/Layers/OnFromTo.csv"
# On-From-To list column containing the "On" street
oft_on_street_col = 'On'
# On-From-To list column containing the "From" street
oft_on_street_col = 'From'
# On-From-To list column containing the "To" street
oft_on_street_col = 'To'


In [ ]:

# Street layer file name (entire street network)
streets_fn = "N:/GIS/_2023/Layers/CDOT Street Centerlines/CNECT_StreetBase_Wards_ComAreas.gpkg"
# street layer column containing the "on" street
streets_on_street_col = 'On_Street'
# street layer column containing the "from" street

In [ ]:
df_oft = pd.read_csv(oft_fn)
df_oft

In [ ]:
gdf_streets = gpd.read_file(streets_fn)
gdf_streets

In [ ]:
def get_street(gdf_all_streets:gpd.GeoDataFrame,
               street_name:str, 
               street_type:str, 
               street_name_col='STREET_NAM', 
               street_type_col='STREET_TYP'):
    street_filter = gdf_all_streets[street_name_col].str.upper() == street_name.upper() 
    type_filter = gdf_all_streets[street_type_col].str.upper() == street_type.upper()
    return gdf_all_streets.loc[street_filter & type_filter]

get_street(gdf_streets,'madison', 'st').explore()

In [ ]:
# Get intersecting points between two geodataframes
# https://gis.stackexchange.com/questions/395315/shapely-coordinate-sequence-to-geodataframe

def get_intersections(street_a:gpd.GeoDataFrame, street_b:gpd.GeoDataFrame):
    return gpd.overlay(a,b, how='intersection', keep_geom_type=False)

a = get_street(gdf_streets, 'chicago', 'ave')
b = get_street(gdf_streets, 'cicero', 'Ave')
get_intersections(a,b).explore()



In [ ]:
# Get nearest segments to each other from two separate geodataframes
# From https://gis.stackexchange.com/questions/222315/finding-nearest-point-in-other-geodataframe-using-geopandas :

# Here is a helper function that will return the distance and 'Name' of the nearest neighbor in gpd2 from each point in gpd1. 
# It assumes both gdfs have a geometry column (of points).

def nearest_segment(gdA, gdB):

#     nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
#     nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    nA = np.array(list(gdA.centroid.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.centroid.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    
    # KEH addition to avoid duplicate column names
    for column_name in gdB_nearest.columns:
        gdB_nearest = gdB_nearest.rename(columns={column_name:column_name+'_B'})
        
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name='dist')
        ], 
        axis=1)
    
    # KH modified to return only the nearest feature (with the minimum distance)
    return gdf.sort_values('dist').iloc[:1]


a = get_street(gdf_streets, 'doty', 'ave')
b = get_street(gdf_streets, '48th', 'pl')

nearest_segment(a,b)

In [ ]:
# Split multiline into segments
def split_segments(shape):
    segments = []
    for pt1,pt2 in zip(shape.coords, shape.coords[1:]):
        segment = LineString([pt1,pt2])
        segments.append(segment)
    return segments

In [ ]:

# Split all features in a GeoDataFrame into their smallest 
# components (individual line segments):
def explode_into_single_segments(gdf:gpd.GeoDataFrame, crs_export="EPSG:3435"):
    
    return_gdf = gpd.GeoDataFrame()
    
    exploded_gdf = gdf.explode(index_parts=True)
       
    for index,row in exploded_gdf.iterrows():
        
        shapes = split_segments(row['geometry'])
        
        # Turn segments into a new dataframe
        gdf_segments = gpd.GeoDataFrame(shapes, columns=('geometry',), crs=crs_export)
        
        # add the segments data to the complete dataframe of all segments
        return_gdf = pd.concat([return_gdf, gdf_segments])
           
    return return_gdf

In [ ]:
##########
# Find extended intersection between two streets, even if they don't actually intersect.
##########

def get_extended_intersection(gdf_a:gpd.GeoDataFrame, gdf_b:gpd.GeoDataFrame):

    if len(gdf_a) == 0 or len(gdf_b) == 0:
        print("Error: empty GeoDataFrame, no streets to analyze.")
        return None
    
    else:

        # explode dataframes into their smallest individual segments
        gdf_a = explode_into_single_segments(gdf_a)
        # print(gdf_a.head(5))
        gdf_b = explode_into_single_segments(gdf_b)
        # print(gdf_b.head(5))

        # Find the nearest individual segments to each other
        nearest_segment_a_to_b = nearest_segment(gdf_a, gdf_b)
        # print(nearest_segment_a_to_b.head(5))
        nearest_segment_b_to_a = nearest_segment(gdf_b, gdf_a)
        # print(nearest_segment_b_to_a.head(5))

        # Extend the nearest segments
        extended_a_to_b = nearest_segment_a_to_b.scale(1000,1000)
        # print(extended_a_to_b)
        extended_b_to_a = nearest_segment_b_to_a.scale(1000,1000)
        # print(extended_b_to_a)


        # Find the point where the extended segments cross
        gdf_intersection = get_intersections(extended_a_to_b, extended_b_to_a)

        # return gdf_intersection
        return (extended_b_to_a, extended_a_to_b)


hubbard = get_street(gdf_streets, "hubbard", "st")
lawler = get_street(gdf_streets, "lawler", "ave")

test = get_extended_intersection(hubbard, lawler)
        
m = test[0].explore()
test[1].explore(m=m)

intersection_test = get_intersections(test[0], test[1])
print(intersection_test)
# intersection_test.explore()

#         # Find centroids of each segment
# #         gdf_street1['centroid'] = gdf_street1.centroid
# #         gdf_street2['centroid'] = gdf_street2.centroid

#         # find distances from street 1 centroids to nearest street 2 centroid:
#         distances_1to2 = ckdnearest(gdf_street1, gdf_street2)
#         # find the segment from street 1 that is closest to street 2:
#         closest_segment_1 = distances_1to2.sort_values('dist').head(1)

#         # find distances from street 2 centroids to nearest street 1 centroid:
#         distances_2to1 = ckdnearest(gdf_street2, gdf_street1)
#         # find the segment from street 2 that is closest to street 1:
#         closest_segment_2 = distances_2to1.sort_values('dist').head(1)

#         # extend both nearest segments
#         ext_1 = closest_segment_1.scale(10000,10000)
#         ext_2 = closest_segment_2.scale(10000,10000)

#         # find the intersection between extended segments
#         intersection_1_2 = gpd.GeoDataFrame(columns=('geometry',), crs="EPSG:3435")
#         geom = ext_1.unary_union.intersection(ext_2.unary_union)
#         intersection_1_2['geometry']=[geom]

#         return intersection_1_2



In [ ]:
###############
##############
# Next:  split multiline segment into separate single lines.  Find the nearest segment and scale it to extend to an intersection.
# Boo.  Explode only turns multilinestrings into linestrings.  Not linestrings into lines.
################
##############
segment = ckdnearest(a,b).iloc[:8]

# exploded = segment.explode()

segment.explore()

print(segment['geometry'])

scaled = segment['geometry'].scale(xfact=50, yfact=50, origin='center')
print(scaled)



In [ ]:
### ChatGPT Assisted version ###

In [2]:
# Import CDOT streets layer
# cdot_centerlines_gdf = gpd.read_file('https://data.cityofchicago.org/api/geospatial/6imu-meau?method=export&format=GeoJSON')
cdot_centerlines_gdf = gpd.read_file('https://data.cityofchicago.org/api/geospatial/6imu-meau?method=export&format=Original')

cdot_centerlines_gdf 

,OBJECTID,FNODE_ID,TNODE_ID,TRANS_ID,PRE_DIR,STREET_NAM,STREET_TYP,SUF_DIR,STREETNAME,L_F_ADD,...,EDIT_TYPE,FLAG_STRIN,EWNS_DIR,EWNS_COORD,CREATE_USE,CREATE_TIM,UPDATE_USE,UPDATE_TIM,SHAPE_LEN,geometry
0,510,10809,16581,127104,S,YALE,AVE,NaN,1782,0,...,NaN,NaN,W,232,EXISTING,1999-01-01,EXISTING,1999-01-01,220.566012,"LINESTRING (1175570.097 1863498.080, 1175577.8..."
1,511,6501,34082,128895,S,COTTAGE GROVE,AVE,NaN,1236,7301,...,NaN,NaN,NaN,0,EXISTING,1999-01-01,EXISTING,1999-01-01,664.774607,"LINESTRING (1182822.668 1856787.427, 1182824.9..."
2,512,15338,22358,142645,S,CAMPBELL,AVE,NaN,1177,10801,...,NaN,NaN,W,2500,EXISTING,1999-01-01,EXISTING,1999-01-01,665.378453,"LINESTRING (1161631.239 1832936.206, 1161634.6..."
3,513,15799,28881,148189,S,SANGAMON,ST,NaN,1696,0,...,NaN,NaN,W,932,EXISTING,1999-01-01,EXISTING,1999-01-01,152.564966,"LINESTRING (1172013.812 1831615.472, 1171905.1..."
4,514,36407,36534,139728,W,118TH,ST,NaN,1823,1933,...,NaN,NaN,S,11800,EXISTING,1999-01-01,EXISTING,1999-01-01,332.691382,"LINESTRING (1165307.502 1826592.692, 1165260.9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56333,71044,37993,610,164905,S,BRAINARD,AVE,NaN,1159,13023,...,Split Street,NaN,NaN,0,ds06027,2016-06-02,ds06027,2016-06-02,267.658450,"LINESTRING (1196162.303 1818891.036, 1196172.1..."
56334,70722,37989,8439,164895,N,MILWAUKEE,AVE,NaN,845,3718,...,Cross Street Change,NaN,NaN,0,ds06027,2016-01-20,ds06027,2016-01-20,291.845926,"LINESTRING (1145920.280 1924326.929, 1145850.8..."
56335,70723,37990,37989,164896,N,KENNETH,AVE,NaN,649,3630,...,Cross Street Change,NaN,W,4432,ds06027,2016-01-20,ds06027,2016-01-21,464.262503,"LINESTRING (1145987.091 1923903.901, 1145962.6..."
56336,19741,33563,16755,121056,W,OHIO,ST,NaN,2452,4701,...,Address Change,NaN,N,600,EXISTING,1999-01-01,ds06027,2015-08-25,622.707971,"LINESTRING (1144876.000 1903573.737, 1144835.2..."


In [3]:
cdot_centerlines_gdf.crs

<Projected CRS: EPSG:3435>
Name: NAD83 / Illinois East (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - Illinois - counties of Boone; Champaign; Clark; Clay; Coles; Cook; Crawford; Cumberland; De Kalb; De Witt; Douglas; Du Page; Edgar; Edwards; Effingham; Fayette; Ford; Franklin; Gallatin; Grundy; Hamilton; Hardin; Iroquois; Jasper; Jefferson; Johnson; Kane; Kankakee; Kendall; La Salle; Lake; Lawrence; Livingston; Macon; Marion; Massac; McHenry; McLean; Moultrie; Piatt; Pope; Richland; Saline; Shelby; Vermilion; Wabash; Wayne; White; Will; Williamson.
- bounds: (-89.27, 37.06, -87.02, 42.5)
Coordinate Operation:
- name: SPCS83 Illinois East zone (US Survey feet)
- method: Transverse Mercator
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [4]:
def scale_linestring(line, scale_length):
    """
    Scale a linestring to a specified length from its midpoint.
    """

    # Calculate the scaling factor
    current_length = line.length
    scaling_factor = scale_length / current_length

    # Scale the line
    midpoint = line.interpolate(0.5, normalized=True)
    scaled_line = affinity.scale(line, xfact=scaling_factor, yfact=scaling_factor, origin=midpoint)

    return scaled_line

In [5]:
def find_nearest_segments(street1_gdf, street2_gdf):
    """
    Find the nearest segment from each street based on their proximity to the other street.
    """
    # Find the segment in street1 that is closest to any point on street2
    min_distance_1 = street1_gdf.distance(street2_gdf.geometry.unary_union).min()
    nearest_segment_1 = street1_gdf[street1_gdf.distance(street2_gdf.geometry.unary_union) == min_distance_1].geometry.iloc[0]

    # Find the segment in street2 that is closest to any point on street1
    min_distance_2 = street2_gdf.distance(street1_gdf.geometry.unary_union).min()
    nearest_segment_2 = street2_gdf[street2_gdf.distance(street1_gdf.geometry.unary_union) == min_distance_2].geometry.iloc[0]
    
    return nearest_segment_1, nearest_segment_2

In [6]:
def extend_segments_to_intersection(segment1, segment2, scale_length=10560):  # 10560 feet = 2 miles
    """
    Extend the two given segments to create an intersection line.
    """
    # Create extended lines from the segments
    extended_line_1 = scale_linestring(segment1, scale_length)
    extended_line_2 = scale_linestring(segment2, scale_length)
    
    # Check intersection of the extended lines
    intersection = extended_line_1.intersection(extended_line_2)
    
    # If they intersect, return the intersection
    if not intersection.is_empty:
        return intersection
    else:
        # If they don't, return both extended lines for visualization
        return extended_line_1, extended_line_2

In [7]:
def get_intersection_point(street1_gdf, street2_gdf, scale_length=2640):  # 2640 ft = 1/2 mile
    """
    Return the intersection point of two streets. If they don't intersect, find the closest features
    and create a virtual intersection by extending the features to the specified scale_length.
    """
    intersection = street1_gdf.geometry.unary_union.intersection(street2_gdf.geometry.unary_union)
    
    # If intersection exists and is a point, return it
    if not intersection.is_empty:
        if intersection.geom_type == "Point":
            return intersection
        elif intersection.geom_type == "MultiPoint":
            return intersection[0]
    
    # If no intersection, find the closest points and create a virtual intersection
    nearest_segment_1, nearest_segment_2 = find_nearest_segments(street1_gdf, street2_gdf)
    
    virtual_intersection = extend_segments_to_intersection(nearest_segment_1, nearest_segment_2)
    
    return virtual_intersection
        
    


In [8]:
def filter_segments_between_points(on_street_gdf, from_intersection, to_intersection):
    """
    Filter the on_street segments based on the orientation of the line formed by the intersections.
    """
    # Determine the orientation of the intersection line
    delta_x = abs(to_intersection.x - from_intersection.x)
    delta_y = abs(to_intersection.y - from_intersection.y)
    
    filtered_segments = []
    
    # If intersection_line is oriented more in the x direction
    if delta_x > delta_y:
        min_x, max_x = sorted([from_intersection.x, to_intersection.x])
        for index, row in on_street_gdf.iterrows():
            midpoint_x = row['geometry'].centroid.x
            if min_x <= midpoint_x <= max_x:
                filtered_segments.append(row['geometry'])
    # If intersection_line is oriented more in the y direction
    else:
        min_y, max_y = sorted([from_intersection.y, to_intersection.y])
        for index, row in on_street_gdf.iterrows():
            midpoint_y = row['geometry'].centroid.y
            if min_y <= midpoint_y <= max_y:
                filtered_segments.append(row['geometry'])
                
    # Convert the list of filtered segments to a GeoDataFrame
    filtered_gdf = gpd.GeoDataFrame(geometry=filtered_segments, crs=on_street_gdf.crs)
    
    return filtered_gdf

In [9]:
    
def extract_street_segments(gdf, on_street, from_street, to_street):
    '''
    Extract the segment of on_street that is between its intersection with from_street and to_street.
    
    on_street, from_street, and to_street are strings representing cleaned official street names found
    in the gdf.  (For example, "Madison St")
    
    gdf is the C*NECT Street Centerline file in a
    GeoDataFrame, containing the fields "On_Street", "From_Street", and "To_Street".
    '''

    # # Filter the GeoDataFrame for the given streets, ignoring case 
    # on_street_gdf = gdf[gdf['On_Street'].str.lower() == on_street.lower()]
    
    # from_street_gdf = gdf[gdf['From_Street'].str.lower() == from_street.lower()]         
    
    # to_street_gdf = gdf[gdf['To_Street'].str.lower() == to_street.lower()]
                        
    # Filter the GeoDataFrame for the given streets, ignoring case 
    ##Old version for CDOT map base layer - requires separate street_nam and 
    # street_typ fields. The arguments for the extract_street_segment file need 
    # to be tuples to use this version. ('Madison', 'St'))

    on_street_gdf = gdf[(gdf['STREET_NAM'].str.lower() == on_street[0].lower()) & 
                        (gdf['STREET_TYP'].str.lower() == on_street[1].lower())]
    
    from_street_gdf = gdf[(gdf['STREET_NAM'].str.lower() == from_street[0].lower()) & 
                          (gdf['STREET_TYP'].str.lower() == from_street[1].lower())]
    
    to_street_gdf = gdf[(gdf['STREET_NAM'].str.lower() == to_street[0].lower()) & 
                        (gdf['STREET_TYP'].str.lower() == to_street[1].lower())]
    
    # Get the intersection points
    on_from_point = get_intersection_point(on_street_gdf, from_street_gdf)
    on_to_point = get_intersection_point(on_street_gdf, to_street_gdf)
    
    # Filter the segments based on the orientation of the line formed by the intersections
    filtered_segments_gdf = filter_segments_between_points(on_street_gdf, on_from_point, on_to_point)
    
    return filtered_segments_gdf

# # Test the combined function
# result_gdf = extract_street_segment(('Madison', 'St'), ('Halsted', 'St'), ('Ashland', 'Ave'))
# result_gdf



In [10]:
## Run Code ##


In [11]:

# archer_test = extract_street_segments(cdot_centerlines_gdf, ('Archer',  'Ave'), ('St Louis', 'Ave'), ('40th', 'st'))

# madison_test = extract_street_segments(cdot_centerlines_gdf, ('madison',  'st'), ('clark', 'st'), ('michigan', 'ave'))


on_street = ('Madison', 'St')
from_street = ('Clark', 'St')
to_street = ('lawndale', 'ave')

test_gdf = extract_street_segments(cdot_centerlines_gdf, on_street, from_street, to_street)

test_gdf.explore()
